In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("/content/drive/MyDrive/PAYTM_historical_data.csv")
data

,Date,Open,High,Low,Close,Volume
0,2022-08-01T09:15:00+05:30,708.00,713.90,707.45,712.75,47041
1,2022-08-01T09:16:00+05:30,713.40,714.00,710.30,711.90,19707
2,2022-08-01T09:17:00+05:30,711.05,712.10,711.00,711.30,14889
3,2022-08-01T09:18:00+05:30,712.00,712.65,711.20,711.95,11883
4,2022-08-01T09:19:00+05:30,712.35,713.55,712.15,712.55,14532
...,...,...,...,...,...,...
184495,2023-07-31T15:25:00+05:30,644.80,644.80,644.15,644.15,19116
184496,2023-07-31T15:26:00+05:30,644.35,644.70,644.15,644.70,22111
184497,2023-07-31T15:27:00+05:30,644.70,644.70,644.35,644.40,24553
184498,2023-07-31T15:28:00+05:30,644.40,644.65,642.90,644.00,67930


In [4]:
print(data.dtypes)

Date       object
Open      float64
High      float64
Low       float64
Close     float64
Volume      int64
dtype: object


In [5]:
data['Open'] = pd.to_numeric(data['Open'], errors='coerce')
data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
data['High'] = pd.to_numeric(data['High'], errors='coerce')
data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
data['Volume'] = pd.to_numeric(data['Volume'], errors='coerce')
data.fillna(0, inplace=True)

In [9]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score

data['Open'] = pd.to_numeric(data['Open'], errors='coerce')
data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
data['High'] = pd.to_numeric(data['High'], errors='coerce')
data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
data.fillna(0, inplace=True)

# Calculate actual returns based on Close prices
data['Actual_Returns'] = data['Close'].pct_change().fillna(0)

# Define the function to implement the moving average crossover strategy
def moving_average_crossover_strategy(data):
    signals = pd.DataFrame(index=data.index)
    signals["Short_MA"] = data["Close"].rolling(window=50).mean()
    signals["Long_MA"] = data["Close"].rolling(window=200).mean()
    signals["Buy_Signal"] = signals["Short_MA"] > signals["Long_MA"]
    signals["Sell_Signal"] = signals["Short_MA"] < signals["Long_MA"]
    return signals

# Define the function to implement the Bollinger Bands strategy
def bollinger_bands_strategy(data):
    signals = pd.DataFrame(index=data.index)
    signals["Rolling_STD"] = data["Close"].rolling(window=20).std()
    signals["Middle_Band"] = data["Close"].rolling(window=20).mean()
    signals["Upper_Band"] = signals["Middle_Band"] + 2 * signals["Rolling_STD"]
    signals["Lower_Band"] = signals["Middle_Band"] - 2 * signals["Rolling_STD"]
    signals["Buy_Signal"] = data["Close"] < signals["Lower_Band"]
    signals["Sell_Signal"] = data["Close"] > signals["Upper_Band"]
    return signals

# Define the function to implement the RSI mean reversion strategy
def rsi_mean_reversion_strategy(data, rsi_window=14, overbought_threshold=70, oversold_threshold=30):
    signals = pd.DataFrame(index=data.index)

    # Calculate RSI
    delta = data["Close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=rsi_window).mean()
    avg_loss = loss.rolling(window=rsi_window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    # Implement the strategy logic based on RSI
    signals["RSI"] = rsi
    signals["Buy_Signal"] = signals["RSI"] < oversold_threshold
    signals["Sell_Signal"] = signals["RSI"] > overbought_threshold
    return signals

# Define the function to implement the Moving Average Convergence Divergence (MACD) strategy
def macd_strategy(data, short_window=12, long_window=26, signal_window=9):
    signals = pd.DataFrame(index=data.index)

    # Calculate MACD line and Signal line
    data['Short_MA'] = data['Close'].rolling(window=short_window).mean()
    data['Long_MA'] = data['Close'].rolling(window=long_window).mean()
    data['MACD_Line'] = data['Short_MA'] - data['Long_MA']
    data['Signal_Line'] = data['MACD_Line'].rolling(window=signal_window).mean()

    # Implement the strategy logic based on MACD and Signal line crossovers
    signals["Buy_Signal"] = data["MACD_Line"] > data["Signal_Line"]
    signals["Sell_Signal"] = data["MACD_Line"] < data["Signal_Line"]
    return signals

# Define the function to implement the Commodity Channel Index (CCI) strategy
def cci_strategy(data, cci_window=20, overbought_threshold=100, oversold_threshold=-100):
    signals = pd.DataFrame(index=data.index)

    # Calculate Typical Price and Moving Average of Typical Price
    data['Typical_Price'] = (data['High'] + data['Low'] + data['Close']) / 3
    data['MA_Typical_Price'] = data['Typical_Price'].rolling(window=cci_window).mean()

    # Calculate Mean Deviation
    data['Mean_Deviation'] = np.abs(data['Typical_Price'] - data['MA_Typical_Price']).rolling(window=cci_window).mean()

    # Calculate Commodity Channel Index (CCI)
    data['CCI'] = (data['Typical_Price'] - data['MA_Typical_Price']) / (0.015 * data['Mean_Deviation'])

    # Implement the strategy logic based on CCI thresholds
    signals["Buy_Signal"] = data["CCI"] < oversold_threshold
    signals["Sell_Signal"] = data["CCI"] > overbought_threshold
    return signals


# Define the function to implement the Average Directional Index (ADX) strategy
def adx_strategy(data, adx_window=14, adx_threshold=25):
    signals = pd.DataFrame(index=data.index)

    # Calculate True Range
    data['HL'] = data['High'] - data['Low']
    data['HC'] = np.abs(data['High'] - data['Close'].shift(1))
    data['LC'] = np.abs(data['Low'] - data['Close'].shift(1))
    data['TR'] = data[['HL', 'HC', 'LC']].max(axis=1)

    # Calculate Directional Movement
    data['DM_plus'] = np.where((data['High'] - data['High'].shift(1)) > (data['Low'].shift(1) - data['Low']), data['High'] - data['High'].shift(1), 0)
    data['DM_minus'] = np.where((data['Low'].shift(1) - data['Low']) > (data['High'] - data['High'].shift(1)), data['Low'].shift(1) - data['Low'], 0)

    # Calculate Smoothed Directional Movement
    data['Smooth_DM_plus'] = data['DM_plus'].rolling(window=adx_window).mean()
    data['Smooth_DM_minus'] = data['DM_minus'].rolling(window=adx_window).mean()

    # Calculate Directional Indexes (DI+ and DI-)
    data['DI_plus'] = (data['Smooth_DM_plus'] / data['TR']) * 100
    data['DI_minus'] = (data['Smooth_DM_minus'] / data['TR']) * 100

    # Calculate Average Directional Index (ADX)
    data['DX'] = (np.abs(data['DI_plus'] - data['DI_minus']) / (data['DI_plus'] + data['DI_minus'])) * 100
    data['ADX'] = data['DX'].rolling(window=adx_window).mean()

    # Implement the strategy logic based on ADX threshold
    signals["Buy_Signal"] = data["ADX"] > adx_threshold
    signals["Sell_Signal"] = data["ADX"] <= adx_threshold
    return signals

# Define the function to implement the On-Balance Volume (OBV) indicator strategy
def obv_strategy(data):
    signals = pd.DataFrame(index=data.index)

    obv_values = []
    for idx, row in data.iterrows():
        if row['Actual_Returns'] > 0:
            obv_values.append(row['Volume'])
        else:
            obv_values.append(-row['Volume'])

    data['OBV'] = obv_values

    signals["Buy_Signal"] = data["OBV"].diff() > 0
    signals["Sell_Signal"] = data["OBV"].diff() < 0

    # Handle NaN values in the first row
    signals["Buy_Signal"] = signals["Buy_Signal"].fillna(False)
    signals["Sell_Signal"] = signals["Sell_Signal"].fillna(False)

    return signals

# Define the function to implement the Average True Range (ATR) indicator strategy
def atr_strategy(data, atr_window=14, atr_threshold=0.01):
    signals = pd.DataFrame(index=data.index)

    # Calculate True Range (TR)
    data['HL'] = data['High'] - data['Low']
    data['HC'] = abs(data['High'] - data['Close'].shift(1))
    data['LC'] = abs(data['Low'] - data['Close'].shift(1))
    data['TR'] = data[['HL', 'HC', 'LC']].max(axis=1)

    # Calculate Average True Range (ATR)
    data['ATR'] = data['TR'].rolling(window=atr_window).mean()

    # Implement the strategy logic based on ATR threshold
    signals["Buy_Signal"] = data["ATR"] > atr_threshold
    signals["Sell_Signal"] = data["ATR"] <= atr_threshold
    return signals

# Define the function to implement the Stochastic Oscillator strategy
def stochastic_oscillator_strategy(data, k_period=14, d_period=3, overbought_threshold=80, oversold_threshold=20):
    signals = pd.DataFrame(index=data.index)

    # Calculate %K and %D
    data['L14'] = data['Low'].rolling(window=k_period).min()
    data['H14'] = data['High'].rolling(window=k_period).max()
    data['%K'] = ((data['Close'] - data['L14']) / (data['H14'] - data['L14'])) * 100
    data['%D'] = data['%K'].rolling(window=d_period).mean()

    # Implement the strategy logic based on %K and %D thresholds
    signals["Buy_Signal"] = data["%K"] < oversold_threshold
    signals["Sell_Signal"] = data["%K"] > overbought_threshold
    return signals


# Calculate predicted returns based on strategy signals
def calculate_predicted_returns(signals):
    predicted_returns = signals["Buy_Signal"].astype(int) - signals["Sell_Signal"].astype(int)
    return predicted_returns


# Define a dictionary of strategy names and their corresponding functions
strategies = {
    "Moving Average Crossover": moving_average_crossover_strategy,
    "Bollinger Bands": bollinger_bands_strategy,
    "RSI mean reversion": rsi_mean_reversion_strategy,
    "Moving Average Convergence Divergence (MACD) strategy": macd_strategy,
    "Commodity Channel Index":cci_strategy,
    "Average Directional Index":adx_strategy,
    "On Balance Volume Indicator":obv_strategy,
    "Average True Range":atr_strategy,
    "Stochastic Oscillator":stochastic_oscillator_strategy,

}

# Iterate through each strategy and evaluate
for strategy_name, strategy_func in strategies.items():
    # Implement the strategy
    signals = strategy_func(data)

    # Calculate predicted returns
    predicted_returns = calculate_predicted_returns(signals)

    # Calculate profit/loss based on signals
    profits_losses = data['Actual_Returns'] * predicted_returns

    # Calculate profit percentage
    initial_investment = 10000  # Adjust this as needed
    final_value = initial_investment + profits_losses.sum()
    profit_percentage = ((final_value - initial_investment) / initial_investment) * 100

    # Calculate evaluation metrics
    # mae = mean_absolute_error(data['Actual_Returns'].fillna(0), predicted_returns.fillna(0))
    accuracy = accuracy_score(data['Actual_Returns'] > 0, predicted_returns > 0)
    precision = precision_score(data['Actual_Returns'] > 0, predicted_returns > 0)
    recall = recall_score(data['Actual_Returns'] > 0, predicted_returns > 0)
    f1 = f1_score(data['Actual_Returns'] > 0, predicted_returns > 0)

    # Print the metrics and profit percentage for each strategy
    print(f"Strategy: {strategy_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"Profit Percentage: {profit_percentage:.2f}%")
    print("\n")

strategy_results= [
    {
        "Strategy":" Moving Average Crossover",
        "Accuracy": 0.4995,
        "Precision": 0.4416,
        "Recall": 0.4747,
        "F1-Score": 0.4575,
        "Profit Percentage": 0.02,
    },
    {
        "Strategy": "Bollinger Bands",
        "Accuracy": 0.5098,
        "Precision": 0.0727,
        "Recall": 0.0087,
        "F1-Score": 0.0156,
        "Profit Percentage": -0.25,
    },
    {
        "Strategy": "RSI mean reversion",
        "Accuracy": 0.5095,
        "Precision": 0.2838,
        "Recall": 0.0677,
        "F1-Score": 0.1093,
        "Profit Percentage": -0.20,

    },
    {
        "Strategy": "Moving Average Convergence Divergence (MACD) strategy",
        "Accuracy": 0.5085,
        "Precision": 0.4532,
        "Recall": 0.5105,
        "F1-Score": 0.4802,
        "Profit Percentage": 0.09,
    },
    {
        "Strategy": "Commodity Channel Index",
        "Accuracy": 0.5017,
        "Precision": 0.2876,
        "Recall": 0.0817,
        "F1-Score": 0.1272,
        "Profit Percentage": -0.23,
    },
    {
        "Strategy": "Average Directional Index",
        "Accuracy": 0.4755,
        "Precision": 0.4466,
        "Recall": 0.7504,
        "F1-Score": 0.5599,
        "Profit Percentage": 0.00,
    },
    {
        "Strategy": "On Balance Volume Indicator",
        "Accuracy": 0.7733,
        "Precision": 0.7175,
        "Recall": 0.8086,
        "F1-Score": 0.7603,
        "Profit Percentage": 0.76,
    },
    {
        "Strategy": "Average True Range",
        "Accuracy": 0.4446,
        "Precision": 0.4446,
        "Recall": 0.9999,
        "F1-Score": 0.6155,
        "Profit Percentage": 0.00,
    },
    {
        "Strategy": "Stochastic Oscillator",
        "Accuracy": 0.4245,
        "Precision": 0.1709,
        "Recall": 0.0764,
        "F1-Score": 0.1056,
        "Profit Percentage": -0.43,

    }
]

# Define weights for evaluation metrics
weights = {
    "Accuracy": 0.2,
    "Precision": 0.2,
    "Recall": 0.2,
    "F1-Score": 0.2,
    "Profit Percentage": 0.2
}


# Calculate overall scores for each strategy based on weighted metrics
for strategy in strategy_results:
    overall_score = (
        strategy["Accuracy"] * weights["Accuracy"] +
        strategy["Precision"] * weights["Precision"] +
        strategy["Recall"] * weights["Recall"] +
        strategy["F1-Score"] * weights["F1-Score"] +
        strategy["Profit Percentage"] * weights["Profit Percentage"]
    )
    strategy["Overall_Score"] = overall_score

# Rank strategies based on overall scores
sorted_strategies = sorted(strategy_results, key=lambda x: x["Overall_Score"], reverse=True)

# Print strategy rankings
print("Strategy Rankings:")
for rank, strategy in enumerate(sorted_strategies, start=1):
    strategy_name = strategy["Strategy"]
    overall_score = strategy["Overall_Score"]
    print(f"{rank}. {strategy_name} - Overall Score: {overall_score:.4f}")

Strategy: Moving Average Crossover
Accuracy: 0.4995
Precision: 0.4416
Recall: 0.4747
F1-Score: 0.4575
Profit Percentage: 0.02%


Strategy: Bollinger Bands
Accuracy: 0.5098
Precision: 0.0727
Recall: 0.0087
F1-Score: 0.0156
Profit Percentage: -0.25%


Strategy: RSI mean reversion
Accuracy: 0.5095
Precision: 0.2838
Recall: 0.0677
F1-Score: 0.1093
Profit Percentage: -0.20%


Strategy: Moving Average Convergence Divergence (MACD) strategy
Accuracy: 0.5085
Precision: 0.4532
Recall: 0.5105
F1-Score: 0.4802
Profit Percentage: 0.09%


Strategy: Commodity Channel Index
Accuracy: 0.5017
Precision: 0.2876
Recall: 0.0817
F1-Score: 0.1272
Profit Percentage: -0.23%


Strategy: Average Directional Index
Accuracy: 0.4755
Precision: 0.4466
Recall: 0.7504
F1-Score: 0.5599
Profit Percentage: 0.00%


Strategy: On Balance Volume Indicator
Accuracy: 0.7733
Precision: 0.7175
Recall: 0.8086
F1-Score: 0.7603
Profit Percentage: 0.76%


Strategy: Average True Range
Accuracy: 0.4446
Precision: 0.4446
Recall: 0.999